In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as opt
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
from heapq import heappush, heappop
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

import evaluation
import data_loader
# from model import MUD
import pdb

In [19]:
params = dict()
params['lr'] = 5e-4
params['batch_size'] = 64
params['epoch_limit'] = 12
params['w_decay'] = 0.1
params['negNum_test'] = 1000
params['epsilon'] = 1e-2
params['negNum_train'] = 4
params['l_size'] = 16
params['gpu']= False


params['lr_cf'] = 1e-1
# params['negNum_train'] = 4
# params["negNum_test"] = 1000
params['epoch_limit_cf'] = 20
params['w_decay_cf'] = 0
# params['batch_size'] = 64
params['gpu'] = False
params['l_size_cf'] = 16
# params['epsilon'] = 0.01


In [3]:
category = 'Baby'
print('Start loading data...')
train, val, test = data_loader.read_data(category)
item_price = data_loader.get_price(category)
item_related = data_loader.get_related(category)
distribution = data_loader.get_distribution(category)
trainset = data_loader.TransactionData(train, item_related, \
            item_price, distribution)
valset = data_loader.UserTransactionData(val, item_price, \
            trainset.itemNum, trainset.userHist)
testset = data_loader.UserTransactionData(test, item_price, \
            trainset.itemNum, trainset.userHist)
avg_rating = trainset.get_avgRating()
print('Finish loading data. Average rating score of training set: %.2f' %avg_rating)

Start loading data...
Finish loading data. Average rating score of training set: 4.20


In [4]:
class MF(nn.Module):
    """
        - userLen: the number of users
        - itemLen: the number of items
        - params: the parameters dict used for constructing model
            - l_size: latent dimension size
            - gpu: True/False, whether using GPU
            
    """
    def __init__(self, userLen, itemLen, avg_rating, params):
        super(MF, self).__init__()
        self.userNum = userLen
        self.itemNum = itemLen
        self.params = params
        if 'gpu' in params and params['gpu'] == True:
            self.device = 'cuda:0'
        else:
            self.device = 'cpu'

        l_size = params['l_size_cf']
        
        """
            Initialize  global bias,
                        user bias,
                        item bias,
                        user embedding,
                        item embedding
        """
        self.globalBias = avg_rating.to(self.device)
        
        self.uBias = nn.Embedding(userLen,1).to(self.device)
        self.uBias.weight.data = torch.zeros_like(self.uBias.weight.data)
        
        self.itemBias = nn.Embedding(itemLen,1).to(self.device)
        self.itemBias.weight.data = torch.zeros_like(self.itemBias.weight.data)
        
#         a = math.sqrt(2.0 / params['l_size'])
        self.uEmbed = nn.Embedding(userLen, l_size).to(self.device)
        self.uEmbed.weight.data.uniform_(-1, 1)
        
        self.itemEmbed = nn.Embedding(itemLen, l_size).to(self.device)
        self.itemEmbed.weight.data.uniform_(-1, 1)
    
    def forward(self, users, items):
        uE = self.uEmbed(users)
#         print('uE:',uE.shape)
        uB = self.uBias(users)
#         print('uB:',uB.shape)
        iE = self.itemEmbed(items)
#         print('iE:',iE.shape)
        iB = self.itemBias(items)
#         print('iB:',iB.shape)
#         gB = self.globalBias.expand(users.shape[0],1)
#         print('mul(uE, iE):',(torch.mul(uE, iE).sum(1)).shape)
        score = self.globalBias + uB + iB + torch.mul(uE, iE).sum(1).view(-1,1)
#         print((self.globalBias + uB + iB).shape)
#         print(((uE*iE).sum(1)).shape)
#         score = (self.globalBias + uB + iB).view(-1) + (uE*iE).sum(1)
#         score = (uE*iE).sum(1)
#         print(score.shape)
        return score
        

In [5]:
print("Start training the rating model...")
Rating = MF(userLen = trainset.userNum, itemLen = trainset.itemNum,\
         avg_rating = avg_rating, params = params)
optimizer_rating = opt.SGD(Rating.parameters(), lr = params['lr_cf'], \
        weight_decay = params['w_decay_cf'])

criterion_rating = nn.MSELoss()


trainset.set_negN(params['negNum_train'])
trainLoader = DataLoader(trainset, batch_size = params['batch_size'], \
                        shuffle = True, num_workers = 0)
valset.set_negN(params['negNum_test'])
valLoader = DataLoader(valset, batch_size = 1, \
                        shuffle = True, num_workers = 0)
testset.set_negN(params['negNum_test'])
testLoader = DataLoader(testset, batch_size = 1, \
                        shuffle = True, num_workers = 0)

epsilon = params['epsilon']
epoch = 0
error = np.float('inf')

Start training the rating model...


In [6]:
runningLoss = []
printLoss = []
epoch = 0
while epoch < params['epoch_limit_cf']:
    epoch += 1
    L = len(trainLoader.dataset)
    pbar = tqdm(total = L)
    for i, batchData in enumerate(trainLoader):
        users = torch.LongTensor(batchData['user'])#.to(Rating.device)
#         print(users.shape)
        items = torch.LongTensor(batchData['item'])#.to(Rating.device)
#         print(items.shape)
        pre_r = Rating.forward(users, items).view(-1)
#         print(pre_r.shape)
        r = torch.FloatTensor(batchData['rating'])#.to(Rating.device)
#         print(r)

        loss = criterion_rating(pre_r,r)
        optimizer_rating.zero_grad()
#         print(loss)
        runningLoss.append(loss.item())
        if (i+1) % 50 == 0:
            printLoss.append(np.mean(np.array(runningLoss[-50:])))
        if (i+1) >= 50:
            pbar.set_postfix({'loss' : '{0:1.5f}'.format(np.mean(np.array(runningLoss[-50:])))})

        loss.backward()
        optimizer_rating.step()

        pbar.update(users.shape[0])
    pbar.close()
    
# while epoch < params['epoch_limit']:
#     runningLoss = []
#     epoch += 1
#     print("Epoch " + str(epoch) + " training...")
#     L = len(trainLoader.dataset)
#     pbar = tqdm(total=L)
#     for i, batchData in enumerate(trainLoader):
#         optimizer_r.zero_grad()
#         # get input
#         users = torch.LongTensor(batchData['user'])#.to(Rating.device)
#         items = torch.LongTensor(batchData['item'])#.to(Rating.device)
#         pre_r = Rating.forward(users, items)
#         r = torch.FloatTensor(batchData['rating'])#.to(Rating.device)

#         loss = criterion_rating(pre_r, r)
        
#         runningLoss.append(loss.item())
#         if (i+1) >= 50:
#             pbar.set_postfix({'loss' : '{0:1.5f}'.format(np.mean(np.array(runningLoss[-50:])))})
                
#         loss.backward()
#         optimizer_r.step()
        
#         pbar.update(users.shape[0])
#     pbar.close()
#     meanMSE = np.mean(np.array(runningLoss))
#     improvement = np.abs(error - meanMSE)
#     error = meanMSE
#     if improvement <= epsilon:
#         print('pre-train stop early')
#         break


HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

In [7]:
Rating.forward(torch.LongTensor([19853,12437,18452,18713,6165,2471,16993]),
              torch.LongTensor([3819,12559,12559,12559,12559,12559,34058]))

tensor([[3.8567],
        [3.9183],
        [5.4678],
        [3.7482],
        [5.1341],
        [3.4442],
        [3.5773]], grad_fn=<AddBackward0>)

In [20]:
class MUD(nn.Module):
    """docstring for MUD"""
    def __init__(self, userLen, itemLen, distribution, item_price, RMF, params):
        super(MUD, self).__init__()
        self.userNum = userLen
        self.itemNum = itemLen
        self.rating = RMF
        self.params = params
        if 'gpu' in params and params['gpu'] == True:
            self.device = 'cuda:0'
        else:
            self.device = 'cpu'
        l_size = params['l_size']
        self.distribution = torch.tensor(distribution).to(torch.float).to(self.device)
        self.price = torch.tensor(item_price).to(torch.float).to(self.device)

        self.gBias = nn.Embedding(1,1).to(self.device)
        self.uBias = nn.Embedding(userLen,1).to(self.device)
        self.itemBias = nn.Embedding(itemLen,1).to(self.device)
        self.uEmbed = nn.Embedding(userLen, l_size).to(self.device)
        self.itemEmbed = nn.Embedding(itemLen, l_size).to(self.device)
        self.to(self.device)

    def forward(self, users, items):
        gB = self.gBias.weight.data.expand(users.shape[0],1)
#         print('gB: ',gB.shape)
        uE = self.uEmbed(users)
#         print('uE: ',uE.shape)
        uB = self.uBias(users)
#         print('uB: ',uB.shape)
        iE = self.itemEmbed(items)
#         print('iE: ',iE.shape)
        iB = self.itemBias(items)
#         print('iB: ',iB.shape)

        alpha = gB + uB + iB + torch.mul(uE, iE).sum(1).view(-1,1)
#         print('alpha: ',alpha.shape)
        
        with torch.no_grad():
            r = self.rating.forward(users, items)
            tanh_r = torch.tanh(r).view(-1,1)
#             print('tanh_r: ',tanh_r.shape)
        price = self.price[items]
#         print('price: ',price.shape)
        
        u = torch.mul(alpha, tanh_r)
#         print('u: ',u.shape)
        out = 0.5 * torch.div(u.view(-1), torch.sigmoid(price))
#         print('out: ',out.shape)
        return out

    def EU(self, users, items):
        gB = self.gBias.weight.data.expand(users.shape[0],1)
        uE = self.uEmbed(users)
        uB = self.uBias(users)
        iE = self.itemEmbed(items)
        iB = self.itemBias(items)
        
        alpha = gB + uB + iB + torch.mul(uE, iE).sum(1).view(-1,1).expand(users.shape[0],5)
        distribution = self.distribution[items]
        rating = torch.tensor([1,2,3,4,5]).expand(users.shape[0],5).to(torch.float).to(self.device)
        tanh_r = torch.tanh(rating)
        U = torch.log(torch.tensor(2).to(torch.float)) * torch.mul(alpha, tanh_r)
        EU = torch.mul(distribution, U).sum(1)
        return EU

    def UE(self, users, items):
        gB = self.gBias.weight.data.expand(users.shape[0],1)
        uE = self.uEmbed(users)
        uB = self.uBias(users)
        iE = self.itemEmbed(items)
        iB = self.itemBias(items)
        
        alpha = gB + uB + iB + torch.mul(uE, iE).sum(1).view(-1,1)
        distribution = self.distribution[items]
        rating = torch.tensor([1,2,3,4,5]).expand(users.shape[0],5).to(torch.float).to(self.device)
        r_bar = torch.mul(distribution, rating).sum(1)
        tanh_r_bar = torch.tanh(r_bar)
        UE = torch.log(torch.tensor(2).to(torch.float)) * torch.mul(alpha, tanh_r_bar)
        return UE

In [21]:
print("Start loading ROM model...")
model = MUD(userLen = trainset.userNum, itemLen = trainset.itemNum,\
         distribution = distribution, item_price = item_price, \
         RMF = Rating, params = params)
optimizer = opt.SGD(model.parameters(), lr = params['lr'], \
        weight_decay = params['w_decay'])

criterion_MUD = nn.BCELoss()
criterion_risk = nn.MSELoss(reduction = 'sum')


epsilon = params['epsilon']

error = np.float('inf')

trainErrorList = []
valErrorList = []
valHistory = []
explodeTempreture = 3
convergenceTempreture = 3

Start loading ROM model...


In [22]:
print("Starting training ROM model...")
epoch = 0
runningLoss = []
while epoch < params['epoch_limit']:
    epoch += 1
    print("Epoch " + str(epoch) + " training...")
    L = len(trainLoader.dataset)
    pbar = tqdm(total = L)
    for i, batchData in enumerate(trainLoader):
        optimizer.zero_grad()
        # get input
        users = torch.LongTensor(batchData['user']).to(model.device)
#         print(users.shape)
        items = torch.LongTensor(batchData['item']).to(model.device)
#         print(items.shape)
        negItems = torch.LongTensor(batchData['negItem']).reshape(-1).to(model.device)
#         print(negItems.shape)

        nusers = users.view(-1,1) 
        nusers = nusers.expand(nusers.shape[0], params['negNum_train']).reshape(-1)
#         print(nusers.shape)

        pOut = model.forward(users,items)#.view(-1)
#         print(pOut.shape)

        nOut = model.forward(nusers, negItems)#.view(-1)
#         nOut = nOut.reshape(-1,params["negNum_train"])
#         print(nOut.shape)

        totalOut = torch.cat((pOut,nOut))
#         print (totalOut.shape)
        
        target = torch.FloatTensor([1]*len(pOut)+[0]*len(nOut))
#         print(target.shape)
        
        m = nn.Sigmoid()
        loss = criterion_MUD(m(totalOut),target)
        runningLoss.append(loss.item())
#         print(loss)
        loss.backward()
        optimizer.step()
        if (i+1) >= 50:
            pbar.set_postfix({'loss' : '{0:1.5f}'.format(np.mean(np.array(runningLoss[-50:])))})

        pbar.update(users.shape[0])
    pbar.close()

    print("Epoch " + str(epoch) + " training risk...")
    pbar = tqdm(total = L)
    for i, batchData in enumerate(trainLoader):
        optimizer.zero_grad()
        users = torch.LongTensor(batchData['user']).to(model.device)
        items = torch.LongTensor(batchData['user']).to(model.device)

        eu = model.EU(users,items)
        ue = model.UE(users,items)

        loss = criterion_risk(ue, eu)
        loss.backward()
        optimizer.step()
        pbar.update(users.shape[0])
    pbar.close()

    #validation
    print("Epoch " + str(epoch) + " validating...")
    with torch.no_grad():
        L = len(valLoader.dataset)
        pbar = tqdm(total = L)
#         model.eval()
        scoreDict = dict()
        for i, batchData in enumerate(valLoader):
            if i > 1000:
                  break
            user = torch.LongTensor(batchData['user'])#.to(model.device)
            posItems = torch.LongTensor(batchData['posItem'])#.to(model.device)
            negItems = torch.LongTensor(batchData['negItem'])#.to(model.device)
            budget = torch.FloatTensor(batchData['budget'])#.to(model.device)
            posPrices = torch.FloatTensor(batchData['posPrice'])#.to(model.device)
            negPrices = torch.FloatTensor(batchData['negPrice'])#.to(model.device)

            items = torch.cat((posItems, negItems),1).view(-1)
            prices = torch.cat((posPrices, negPrices),1).view(-1)
            users = user.expand(items.shape[0])

            out = model.forward(users,items)
            scoreHeap = list()
            for j in range(out.shape[0]):
                gt = False
                if j < posItems.shape[1]:
                    gt = True
#                 if prices[j] > budget:
#                     heappush(scoreHeap, (100, (0 + items[j].cpu().numpy(), gt)))
#                 else:
                heappush(scoreHeap, (1 - out[j].cpu().numpy(), (0 + items[j].cpu().numpy(), gt)))
            scores = list()
            candidate = len(scoreHeap)
            for k in range(candidate):
                scores.append(heappop(scoreHeap))
            pbar.update(1)
            scoreDict[user[0]] = (scores, posItems.shape[1])
        pbar.close()

    valHistory.append(evaluation.ranking_performance(scoreDict,10))
#     valError = 1 - valHistory[-1]["avg_ndcg"][0]
#     valErrorList.append(valError)
#     improvement = np.abs(error - valError)
#     error = valError
#     if improvement < epsilon:
#         print("stop early")
#         break



Starting training ROM model...
Epoch 1 training...


HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

Epoch 1 training risk...


HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

Epoch 1 validating...


...

Epoch 12 training risk...


HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

Epoch 12 validating...


HBox(children=(IntProgress(value=0, max=23894), HTML(value='')))

	Precision@: {1:0.002997002997; 5: 0.0011988011988; 10: 0.0011988011988}
	Recall@: {1:0.00137362637363; 5: 0.00387112887113; 10: 0.00656288156288}
	NDCG@: {1:0.002997002997; 5: 0.00372736851716; 10: 0.0046707578539}


In [32]:
# test
print("starting test...")
L = len(testLoader.dataset)
pbar = tqdm(total = L)
scoreDict = dict()
with torch.no_grad(): 
    for i, batchData in enumerate(testLoader):
#         if i>1000:
#             break
        user = torch.LongTensor(batchData['user']).to(model.device)
        posItems = torch.LongTensor(batchData['posItem']).to(model.device)
        negItems = torch.LongTensor(batchData['negItem']).to(model.device)
        budget = torch.FloatTensor(batchData['budget']).to(model.device)
        posPrices = torch.FloatTensor(batchData['posPrice']).to(model.device)
        negPrices = torch.FloatTensor(batchData['negPrice']).to(model.device)

        items = torch.cat((posItems, negItems),1).view(-1)
        prices = torch.cat((posPrices, negPrices),1).view(-1)
        users = user.expand(items.shape[0])

        out = model.forward(users,items)
        scoreHeap = list()
        for j in range(out.shape[0]):
            gt = False
            if j < posItems.shape[1]:
                gt = True
#             if prices[j] > budget:
#                 heappush(scoreHeap, (1000, (0 + items[j].cpu().numpy(), gt)))
#             else:
#                 heappush(scoreHeap, (1 - out[j].cpu().numpy(), (0 + items[j].cpu().numpy(), gt)))
            heappush(scoreHeap, (1 - out[j].cpu().numpy(), (0 + items[j].cpu().numpy(), gt)))
        scores = list()
        candidate = len(scoreHeap)
        for k in range(candidate):
            scores.append(heappop(scoreHeap))
        pbar.update(1)
        scoreDict[user[0]] = (scores, posItems.shape[1])
pbar.close()
testResult = evaluation.ranking_performance(scoreDict,10)

starting test...


HBox(children=(IntProgress(value=0, max=23894), HTML(value='')))

	Precision@: {1:0.00209257554198; 5: 0.00147317318155; 10: 0.00141458106638}
	Recall@: {1:0.00156126215702; 5: 0.00545997950786; 10: 0.0103926450771}
	NDCG@: {1:0.00209257554198; 5: 0.00407045459454; 10: 0.00576478077336}


In [27]:
len(scoreDict)

23894

In [11]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input), target)
output.backward()

In [12]:
input

tensor([-0.6713,  0.9573,  0.6167], requires_grad=True)

In [13]:
target

tensor([1., 0., 0.])

In [14]:
criterion_MUD(m(input),target)

tensor(1.1382, grad_fn=<BinaryCrossEntropyBackward>)